# ResNet50 Trainen
In deze notebook gaan we een pre-trained ResNet50 model door trainen met onze eigen dataset en klassen.

In [2]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.datasets import ImageFolder
from torchvision import datasets, models, transforms
from torchvision.models import resnet50, ResNet50_Weights
from torch.utils.data import DataLoader, random_split

from PIL import Image, ImageOps
import matplotlib.pyplot as plt
from IPython.display import FileLink, FileLinks

from collections import defaultdict
from fractions import Fraction

import random
import time

from transformer import ResizeWithRandomRotation

### Data Understanding

Deze notebook richt zich alleen op het uitvoeren van het train proces, hieronder vind je code op een blik te kunnen doen op de waardes van de dataset.

In [5]:
FileLink('Data Understanding.ipynb')

E:\Studie\Stage\Computer-Vision-Showcase-SUPERP\code\Data Understanding.ipynb

#### Image Processing
De aanpassingen die we hier op de dataset gaan doen staan in onderstaande notebook. Dit is ook waar we het gemiddelde en de standaardafwijking berekenen die we later bij normalize gebruiken.

In [7]:
FileLink('Image Processing.ipynb')

E:\Studie\Stage\Computer-Vision-Showcase-SUPERP\code\Image Processing.ipynb

### Model Trainen

In deze notebook gaan we dieper in op verschillende mogelijkheden tijdens het door trainen van een pre-trained image classification model. [Waarom pre-trained en waar kan je pre-trained models vinden](https://www.reduct.store/blog/pre-trained-models-computer-vision)

#### Data laden

__Let op:__ Voor je het een model kan trainen moet je eerst een eigen dataset toevoegen of maak gebruik van de `selfmade_data`. *(voor een snelle test of het werkt is deze dataset alleen iets te groot)*

In [12]:
# path naar data folder
data_dir = '..//images//selfmade_data'

In [13]:
# Aantal klasse in de dataset
num_classes = 6

##### Standaard instellingen

In [15]:
# Default Instellingen
img_size = 224
batch_size = 32
num_epochs = 50
learning_rate = 0.001

### Error handeling
Tijdens het trainprocess is het mogelijk onverwachte problemen tegen te komen zoals het verdwijnen van afbeeldingen uit je dataset.
In onderstaande geval werden afbeeldingen tijdens het trainen van het model corrupt geraakt waardoor het train process een foutmelding gaf.

`UnidentifiedImageError: cannot identify image file <_io.BufferedReader name='E:\\Studie\\Stage\\combined\\Banaan\\56.jpeg'>`

*(In dit geval raakte de scraped images corrupt)*

### Check Corrupted
Onderstaande code checkt of afbeeldingen voor het train process al corrupt zijn.

In [18]:
root_dir = data_dir

# Loop door alle directories en subdirectories
for subdir, dirs, files in os.walk(root_dir):
    for filename in files:
        filepath = os.path.join(subdir, filename)
        try:
            with Image.open(filepath) as img:
                img.verify()  # Verifieert de integriteit zonder te laden
        except (IOError, SyntaxError, Image.UnidentifiedImageError) as e:
            print(f'Probleem met bestand: {filepath} - {e}')

Van te voren zijn afbeeldingen nog niet corrupt maar dit kan wel tijdens het proces gebeuren.

Gelukkig kunnen we deze code gebruiken om de dataloader aan te passen. We neemen de ImageFolder van Pytorch en 'veranderen' de `__getitem__()`. Elke keer als een batch van de afbeeldingen wordt ingeladen wordt elke afbeelding gecheckt tot de afbeelding is ingeladen of een foutmelding opleverd. 

In [20]:
class SafeImageFolder(ImageFolder):
    def __getitem__(self, index):
        while True:
            try:
                return super(SafeImageFolder, self).__getitem__(index)
            except (IOError, OSError, Image.UnidentifiedImageError):
                print(f'Fout bij het laden van bestand bij index {index}, overslaan.')

#### SafeImageFolder: Een robuuste DataLoader - Extra Informatie

De `SafeImageFolder` klasse is een aangepaste versie van de PyTorch `ImageFolder`, ontworpen om robuuster om te gaan met het laden van afbeeldingsdata. Het bevat een verbeterde foutafhandeling die problemen zoals corrupte bestanden tijdens het inladen van afbeeldingen op een veilige manier afvangt.
##### Kernfunctionaliteit:
- **Overerving van ImageFolder**: Neemt alle basisfunctionaliteit van `ImageFolder` over, waardoor het eenvoudig te gebruiken is met bestaande PyTorch data pipelines.
- **Veilig Laden met Foutafhandeling**:
  - De `__getitem__` methode herhaalt het laden van een afbeelding tot het succesvol is of een onherstelbare fout optreedt.
  - Vangt specifieke fouten op zoals `IOError`, `OSError` en `Image.UnidentifiedImageError`, en slaat beschadigde of onleesbare bestanden over met een melding.
##### Praktische Voordelen:
- **Robuuste Data Loading**: Fouten worden opgevangen zowel bij het initialiseren van de dataset als gedurende het trainen, valideren, of testen. Dit betekent dat als een bestand plotseling corrupt raakt, het trainingsproces niet crasht.
- **Betrouwbaarheid**: Verhoogt de betrouwbaarheid van het trainingsproces, vooral nuttig bij het werken met grote datasets of datasets van wisselende kwaliteit.
Gebruik `SafeImageFolder` om ervoor te zorgen dat je modeltraining zonder onderbreking verloopt, zelfs wanneer je te maken hebt met onbetrouwbare data.

### Early Stopping
Om overfitting te voorkomen kan je ook gebruik maken van Early Stopping. Early Stopping is zoals de naam al zegt een class die het train process tussentijds stop kan zetten.  

In [24]:
class EarlyStopping:
    def __init__(self, patience=5, min_delta=0.0, save_path=None):
        self.patience = patience
        self.min_delta = min_delta
        self.best_loss = float('inf')
        self.counter = 0
        self.save_path = save_path  # Pad om het beste model op te slaan

    def __call__(self, val_loss, model):
        if val_loss < self.best_loss - self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
            if self.save_path:
                torch.save(model.state_dict(), self.save_path)  # Model opslaan
        else:
            self.counter += 1

        return self.counter >= self.patience #<--- Ga dit nog nader checken

##### Overfitting

Overfitting is wanneer een model te ver doorgetraind is op een dataset waardoor het resultaat op nieuwe data slechter wordt.
Overfitting is een veelvoorkomend probleem in machine learning waarbij een model te goed presteert op de trainingsdata maar slecht generaliseert naar nieuwe, ongeziene data. 

Hier zijn enkele kenmerkende signalen van overfitting:

- **Hoge Training Prestaties maar Lage Validatie/Test Prestaties**:
Het model behaalt lage foutpercentages of hoge nauwkeurigheid op de trainingsdataset, maar laat hoge foutpercentages of lage nauwkeurigheid zien op de validatie- of testdataset.
- **Toenemende Kloof tussen Training en Validatie Fouten**:
Tijdens het trainen daalt de fout (bijvoorbeeld verlies) op de trainingsset constant, terwijl de validatie- of testfout op een gegeven moment begint te stijgen of stabiliseert en dan verslechtert.
- **Lange Trainingstijd zonder Algemene Verbeteringen**:
Het model blijft beter presteren op de trainingsset bij langere trainingstijd, maar prestaties op de validatie- of testset blijven stagneren of worden slechter.
- **Complex Model met Vergeleken met Data**:
Een model met een hoge capaciteit (bijvoorbeeld diep netwerk met veel parameters) toegepast op een relatief kleine dataset kan overmatig de specifieke details van de trainingsdata leren.
- **Visueel Indicaties**:
Bij het plotten van de verliescurves voor training en validatie, een uiteenlopende visualisatie waarin de validatie curve omhoog gaat terwijl de trainings curve blijft dalen, duidt vaak op overfitting.
- **Ruis Leren**:
Het model begint specifieke ruis of uitzonderlijke patronen in de trainingsset te leren, die niet representatief zijn voor de bredere dataset.

Het herkennen van overfitting is onderdeel van het reguliere proces van het ontwikkelen van machine-learning-modellen. Technieken zoals regularisatie, vroegtijdig stoppen, cross-validatie, en data-augmentatie kunnen helpen overfitting te verminderen en de generaliserende kracht van je model te verbeteren.

### Inladen Data

In [28]:
# ImageNet standaard
mean=[0.485, 0.456, 0.406]
std=[0.229, 0.224, 0.225]

In [29]:
mean = [0.5146, 0.5071, 0.5064]
std = [0.1779, 0.1722, 0.1627]

In [30]:
# Datatransformaties
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    ResizeWithRandomRotation(img_size, 25),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

In [31]:
# Laad de dataset
# full_dataset = datasets.ImageFolder(root=data_dir, transform=transform) <-- de standaard image folder call
full_dataset = SafeImageFolder(root=data_dir, transform=transform)

# Verdeel de dataset in training-, validatie- en testdata
dataset_size = len(full_dataset)
train_size = int(0.7 * dataset_size)
val_size = int(0.15 * dataset_size)
test_size = dataset_size - train_size - val_size

train_data, val_data, test_data = random_split(full_dataset, [train_size, val_size, test_size])

# DataLoaders
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

#### Model Inladen

We geven hier aan dat we het ResNet50 model willen gebruiken met de standaard weights van het model, in het geval je een eigen model zou willen trainen moet je deze structuur eerst zelf opzetten.

In [34]:
# Laad het ResNet-50 model
model = resnet50(weights=ResNet50_Weights.DEFAULT)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, num_classes)

In [35]:
# Eigen model structuur
FileLink('simpel_CNN.ipynb')

E:\Studie\Stage\Computer-Vision-Showcase-SUPERP\code\simpel_CNN.ipynb

##### Change Input Size

Op het moment dat je de resolutie van de input 224x224 wilt houden hoeft de `input_tensor` niet worden aangepast. In andere gevallen is dit hoe je dat toepast.

In [38]:
input_tensor = torch.randn(1, 3, img_size, img_size)

In [39]:
output = model(input_tensor)

In [40]:
print(output.shape)  # Controleer of de uitvoer overeenkomt met het verwachte aantal klassen

torch.Size([1, 6])


##### Device setup

In [42]:
# Verplaats het model naar GPU als dat beschikbaar is
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [43]:
print(device)

cuda


### Model Setup

#### Early Stopping Setup

`min_delta` is de minimale hoeveelheid dat de loss moet verbeteren ten opzichte van de vorige epoch. `patience` is het aantal epochs dat het wachten op deze verbetering maximaal gaat duren.

In [47]:
# Standaard instellingen
patience = 5
min_delta = 0.01

In [48]:
# Initialiseren van EarlyStopping
early_stopping = EarlyStopping(patience=patience, min_delta=min_delta, save_path="ResNet50_best_val_score.pth")

#### Criteria en Loss
In het trainen van computervisie-modellen, zoals CNN's, is het essentieel om een geschikte verliesfunctie (loss function) te kiezen, omdat deze fungeert als een maatstaf voor hoe goed het model presteert. 
- **Criterion**: Dit verwijst naar de verliesfunctie die gekozen is voor het model. In de context van classificatiemodellen voor computervisie, vooral wanneer er meerdere klassen zijn, is `nn.CrossEntropyLoss` een veelgebruikte keuze. Deze verliesfunctie combineert `nn.LogSoftmax` en `nn.NLLLoss` in één enkele stap en is daarom efficiënter en nauwkeuriger voor het berekenen van de verschillen tussen de voorspelde klassen en de werkelijke klassen. Het stuurt het model aan om het logaritmische verschil tussen voorspellingen en waarheidslabel te minimaliseren.
#### Keuzes Onderbouwd voor Optimizer
1. **Adam Optimizer**:
   - Voordelen: Adam (Adaptive Moment Estimation) past leerrates aan voor elke parameter individueel en automatiseert veel van de gewenste eigenschappen van de reguliere Stochastic Gradient Descent (SGD). Het vereist minder fijnafstelling en werkt goed voor een snel convergeren van modellen.
   - Nadelen: Het kan minder goed generaliseren in sommige scenario's, omdat het de neiging heeft om zich sterk aan te passen aan de trainingsdata (potentieel meer vatbaar voor overfitting).
2. **SGD met Momentum en Weight Decay**:
   - **SGD**: Stochastic Gradient Descent is klassiek en krachtig, maar kan traag convergeren zonder extra technieken.
   - **Momentum**: Dit helpt om de convergentiesnelheid te verhogen door ervoor te zorgen dat het model zijn voortgang in een bepaalde richting voortzet, waardoor het leerproces stabieler wordt.
   - **Weight Decay**: Dit voegt L2 regularisatie toe, wat helpt de overfitting te beperken door te straffen voor te grote gewichten.
   - Voordelen: Deze configuratie kan leiden tot betere generalisatieprestaties en is vaak de voorkeur bij grotere datasets of wanneer zo hoog mogelijke nauwkeurigheid gewenst is. Het is wel gevoeliger voor hyperparameterinstellingen (zoals leersnelheid) dan Adam.

De keuze voor de SGD optimizer met momentum en weight decay suggereert een voorkeur voor verfijning en optimalisatie, waarbij het risico van overfitting wordt beperkt en de model algemene prestatie verbeterd. Door het gewicht van parameters onder controle te houden en de richting van de update te stabiliseren, leert het model vaak robuuster en effectiever voor het geven van generaliserende resultaten.

In [50]:
# Loss en optimizer
criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=learning_rate) OLD
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=0.0001)

#### Scheduler

In [52]:
# Initialiseer de scheduler
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

##### Verklaring hyperparameters

`Momentum=0.9` dit maakt het verschil tussen Stochastic Gradient Descent (SGD) en Stochastic Gradient Descent with Momentum (SGDM)
Je zou hier kunnen testen tussen 0.8 en 0.99 maar historisch gezien is 0.9 bewezen effectief bij veel beeldherkenningsproblemen.
Aangezien het train proces al erg lang duurt gaan we dit niet testen.
`weight_decay=0.0001` is om overfitting te voorkomen, dit is een gebruikelijke waarde wat effect zou hebben maar niet te heftig is en andere problemen kan veroorzaken. 
`step_size=7` betekent dat om de 7 epochs de learning rate wordt verlaagd.
`gamma=0.1` is het percentage waarmee de learning rate wordt verlaagd, in dit geval zou dus elke 7 steps de learningrate met 10% worden verminderd.

In [55]:
# Training Loop
for epoch in range(num_epochs):
    start = time.time()
    model.train()
    
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    
    epoch_loss = running_loss / len(train_loader)
    end = time.time()
    
    print(f'Epoch {epoch}/{num_epochs - 1}, Loss: {epoch_loss:.4f}, Tijd epoch: {end - start} seconden")')

    # Validatie
    model.eval()
    val_running_loss = 0.0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_running_loss += loss.item()

    val_loss = val_running_loss / len(val_loader)
    print(f'Validation Loss: {val_loss:.4f}')

    # Scheduler stap na elke epoch  
    scheduler.step()
    
    # Controleer voor early stopping
    # if early_stopping(val_loss):
    if early_stopping(val_loss, model):
        print("Early stopping")
        break

print('Training compleet')

D:\Luuk\anaconda3\lib\site-packages\torch\autograd\graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Epoch 0/49, Loss: 1.2958, Tijd epoch: 191.26768112182617 seconden")
Validation Loss: 0.4058
Epoch 1/49, Loss: 0.1861, Tijd epoch: 188.1691288948059 seconden")
Validation Loss: 0.0608
Epoch 2/49, Loss: 0.0676, Tijd epoch: 191.78909420967102 seconden")
Validation Loss: 0.0338
Epoch 3/49, Loss: 0.0401, Tijd epoch: 191.11213541030884 seconden")
Validation Loss: 0.0251
Epoch 4/49, Loss: 0.0260, Tijd epoch: 191.41813325881958 seconden")
Validation Loss: 0.0258
Epoch 5/49, Loss: 0.0241, Tijd epoch: 190.72707557678223 seconden")
Validation Loss: 0.0167
Epoch 6/49, Loss: 0.0224, Tijd epoch: 188.31207513809204 seconden")
Validation Loss: 0.0112
Epoch 7/49, Loss: 0.0139, Tijd epoch: 189.96711468696594 seconden")
Validation Loss: 0.0139
Epoch 8/49, Loss: 0.0179, Tijd epoch: 190.42907571792603 seconden")
Validation Loss: 0.0159
Epoch 9/49, Loss: 0.0137, Tijd epoch: 189.8340663909912 seconden")
Validation Loss: 0.0152
Epoch 10/49, Loss: 0.0134, Tijd epoch: 189.241055727005 seconden")
Validation Loss

### Model Opslaan

Nadat het model is getraind is het handig om het model op te slaan. Hierbij kunnen we weer gebruik maken van torch om de gewichten van het model te downloaden. Het is ook mogelijk zelf gelijk het model om te zetten naar het de manier hoe je het wilt gebruiken maar vooral voor relatieve beginners raad ik aan dit stap voor stap te doen.

In [58]:
# Opslaan van het model
torch.save(model.state_dict(), 'default_name.pth') # Vergeet niet te hernoemen

    Geef het .pth bestand een logische naam waar het model aan te herkennen is.

# Model Gebruiken

Normaal gesproken hoef je het model niet opnieuw in te laden na het trainen van het model. Echter staat nu het gebruiken van het model in een andere notebook dus hebben we het .pth bestand nodig om niet opnieuw het train proces te hoeven draaien.

In [62]:
FileLink('ResNet50_model_gebruiken.ipynb')

E:\Studie\Stage\Computer-Vision-Showcase-SUPERP\code\ResNet50_model_gebruiken.ipynb